In [2]:
import GPy
import numpy as np

# define the input data and output values
X = np.random.uniform(0, 1, (50, 1))
Y1 = np.sin(X) + np.random.normal(0, 0.1, (50, 1))
Y2 = np.cos(X) + np.random.normal(0, 0.1, (50, 1))
Y3 = Y1 + Y2

# define the kernel functions
kernel1 = GPy.kern.RBF(input_dim=1)
kernel2 = GPy.kern.RBF(input_dim=1)
kernel3 = kernel1 + kernel2

# define the models
model1 = GPy.models.GPRegression(X, Y1, kernel1)
model2 = GPy.models.GPRegression(X, Y2, kernel2)
likelihood_list = [GPy.likelihoods.Gaussian(), GPy.likelihoods.Gaussian()]
model3 = GPy.models.MultioutputGP(X, [Y1, Y2], kernel_list=None, likelihood_list=likelihood_list)
model3.kern = kernel3.copy()

# define the sum constraint
A = np.array([[0, 0, 1]])
b = np.array([Y3.sum()])

# set the constraint on the kernel of the multitask model
kernel3.constrain(np.dot(A, kernel3.param_array), "==", b)

# optimize the models
model1.optimize()
model2.optimize()
model3.optimize()

# make predictions
X_test = np.linspace(0, 1, 100).reshape(-1, 1)
Y1_test, _ = model1.predict(X_test)
Y2_test, _ = model2.predict(X_test)
Y3_test, _ = model3.predict(X_test)

# plot the results
import matplotlib.pyplot as plt

plt.plot(X_test, Y1_test, label='Y1')
plt.plot(X_test, Y2_test, label='Y2')
plt.plot(X_test, Y3_test[:, 0], label='Y3_1')
plt.plot(X_test, Y3_test[:, 1], label='Y3_2')
plt.legend()
plt.show()


AssertionError: 